# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.236131892773                   -0.50    7.0
  2   -7.250290369612       -1.85       -1.38    1.0
  3   -7.251127374536       -3.08       -1.93    4.0
  4   -7.250942002391   +   -3.73       -1.87    3.0
  5   -7.251333035860       -3.41       -2.74    2.0
  6   -7.251337912023       -5.31       -3.11    2.0
  7   -7.251338780582       -6.06       -4.02    2.0
  8   -7.251338793857       -7.88       -4.19    4.0
  9   -7.251338798456       -8.34       -4.76    2.0
 10   -7.251338798635       -9.75       -5.11    2.0
 11   -7.251338798697      -10.21       -5.64    3.0
 12   -7.251338798704      -11.12       -6.26    2.0
 13   -7.251338798704      -12.75       -6.58    3.0
 14   -7.251338798705      -13.44       -7.08    3.0
 15   -7.251338798705      -14.10       -7.46    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0512313015663491
[ Info: Arnoldi iteration step 2: normres = 0.5910475161562285
[ Info: Arnoldi iteration step 3: normres = 0.8400268042546531
[ Info: Arnoldi iteration step 4: normres = 0.2694348189593592
[ Info: Arnoldi iteration step 5: normres = 0.48276803374039623
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.41e-02, 7.29e-02, 4.29e-01, 2.08e-01, 1.63e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2916028750772652
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (5.84e-03, 1.14e-01, 2.21e-01, 1.22e-01, 3.31e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07696784921831189
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.17e-04, 8.68e-03, 1.46e-02, 4.38e-02, 5.59e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11034678917131654
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.03e-0